## 라이브러리

In [36]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [37]:
select_category = 'Drama' # 카테고리를 선택해주세요!! 


title_list = os.listdir(f'../script/{select_category}')

title_list = pd.DataFrame(title_list)
title_list = title_list.rename(columns = {0 : 'title'})

title_list

,title
0,.DS_Store
1,Anne.with.an.E
2,Better.Call.Saul
3,Black.Mirror
4,Breaking.Bad
5,Disenchantment
6,Friends
7,How.to.Get.Away.With.Murder
8,MINDHUNTER
9,Narcos


# 위의 리스트에서 작품 인덱스를 넣으세요

In [38]:
title_num = 13 # 작품 인덱스 넣는 곳

In [41]:
_list = os.listdir(f'../script/{select_category}/' + title_list.loc[title_num]['title'])
file_list = []
for file in _list:
    if file.startswith('.'): continue
    if file[-3:] == 'csv': continue
    file_list.append(file)
file_list

['Stranger.Things.S01E01.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E02.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E03.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E04.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E05.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E06.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E07.WEBRip.Netflix.en[cc].vtt',
 'Stranger.Things.S01E08.WEBRip.Netflix.en[cc].vtt']

In [42]:
title = title_list.loc[title_num]['title']

path = f'../script/{select_category}/' + title_list.loc[title_num]['title']

full_segments = ''

for file_name in file_list:
    print(file_name)
    opened_file = open(f'{path}/{file_name}', encoding='utf8')
    content = opened_file.read()
    segments = content.split('NOTE /SegmentIndex')[1]    # 한줄씩 정리

    full_segments += segments

segments = full_segments.split('\n\n')

Stranger.Things.S01E01.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E02.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E03.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E04.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E05.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E06.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E07.WEBRip.Netflix.en[cc].vtt
Stranger.Things.S01E08.WEBRip.Netflix.en[cc].vtt


In [43]:
display(len(segments), segments[:5])

5769

['',
 '                                                                                                                                                                                          ',
 '\n1\n00:00:07.465 --> 00:00:09.175 position:50.00%,middle align:middle size:80.00% line:84.67% \n<c.bg_transparent>[crickets chirping]</c.bg_transparent>',
 '2\n00:00:49.799 --> 00:00:51.926 position:50.00%,middle align:middle size:80.00% line:84.67% \n<c.bg_transparent>[alarm blaring]</c.bg_transparent>',
 '3\n00:00:52.010 --> 00:00:54.471 position:50.00%,middle align:middle size:80.00% line:84.67% \n<c.bg_transparent>[panting]</c.bg_transparent>']

## 불필요한 부분 제거

In [44]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [45]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:10]

[None,
 None,
 None,
 None,
 None,
 None,
 '10\n00:01:48.233  00:01:52.821 A shadow grows on the wall behind you,\nswallowing you in darkness.',
 '11\n00:01:52.904  00:01:54.864 -It is almost here.\n-What is it?',
 "12\n00:01:54.948  00:01:56.282 What if it's the Demogorgon?",
 "13\n00:01:56.366  00:01:59.410 Oh, Jesus, we're so screwed\nif it's the Demogorgon."]

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [67]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment.strip() != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = ' ' .join(split_segment[3:])
      times.append(script)
      trimmed_segments.append(times)

In [70]:
trimmed_segments[:5]

[['10',
  '00:01:48.233',
  '00:01:52.821',
  'A shadow grows on the wall behind you, swallowing you in darkness.'],
 ['11', '00:01:52.904', '00:01:54.864', '-It is almost here. -What is it?'],
 ['12', '00:01:54.948', '00:01:56.282', "What if it's the Demogorgon?"],
 ['13',
  '00:01:56.366',
  '00:01:59.410',
  "Oh, Jesus, we're so screwed if it's the Demogorgon."],
 ['14', '00:01:59.494', '00:02:00.537', "It's not the Demogorgon."]]

## 데이터 저장

In [71]:
file_name = file_list[0].split('.S01')[0] + '.WEBRip.Netflix.en[cc]'
file_name

'Stranger.Things.WEBRip.Netflix.en[cc]'

In [72]:
with open(f'{path}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [73]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv(f'{path}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

data.head()

,num,start,end,script
0,10,00:01:48.233,00:01:52.821,"A shadow grows on the wall behind you, swallowing you in darkness."
1,11,00:01:52.904,00:01:54.864,-It is almost here. -What is it?
2,12,00:01:54.948,00:01:56.282,What if it's the Demogorgon?
3,13,00:01:56.366,00:01:59.410,"Oh, Jesus, we're so screwed if it's the Demogorgon."
4,14,00:01:59.494,00:02:00.537,It's not the Demogorgon.


In [74]:
data['script'] = data['script'].str.replace('\"', '')

# 쪼개진 문장 합치기
for index, current_row in data.iterrows():
    if current_row['script'][-1] in ['.', '!', '?']:
        continue
    else:
        row_index = index + 1
    while True:
        next_row = data.loc[row_index]
        new_script = data.loc[index, 'script'] + ' ' + next_row['script']
        data.loc[index, 'script'] = new_script
        if next_row['script'][-1] in ['.', '!', '?']:
            data.loc[index, 'end'] = next_row['end']
            break
        else:
            row_index += 1

# 필요없는 토막문장들 제거
data = data.drop_duplicates(subset='end')
data.head(30)

,num,start,end,script
0,10,00:01:48.233,00:01:52.821,"A shadow grows on the wall behind you, swallowing you in darkness."
1,11,00:01:52.904,00:01:54.864,-It is almost here. -What is it?
2,12,00:01:54.948,00:01:56.282,What if it's the Demogorgon?
3,13,00:01:56.366,00:01:59.410,"Oh, Jesus, we're so screwed if it's the Demogorgon."
4,14,00:01:59.494,00:02:00.537,It's not the Demogorgon.
5,15,00:02:00.620,00:02:02.872,An army of troglodytes charge into the chamber!
6,19,00:02:10.630,00:02:12.590,Did you hear that?
7,20,00:02:12.674,00:02:14.092,That... that sound?
8,21,00:02:14.175,00:02:16.845,Boom... boom...
9,23,00:02:18.763,00:02:20.640,"That didn't come from the troglodytes. No, that..."


In [75]:
data.to_csv(f'{path}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{file_name}.csv')

data

,start,end,script
0,00:01:48.233,00:01:52.821,"A shadow grows on the wall behind you, swallowing you in darkness."
1,00:01:52.904,00:01:54.864,-It is almost here. -What is it?
2,00:01:54.948,00:01:56.282,What if it's the Demogorgon?
3,00:01:56.366,00:01:59.410,"Oh, Jesus, we're so screwed if it's the Demogorgon."
4,00:01:59.494,00:02:00.537,It's not the Demogorgon.
...,...,...,...
3590,00:51:56.613,00:51:59.658,"Mmm. Hey, Mom. Did Will tell you about, uh, the game?"
3591,00:51:59.741,00:52:00.909,"-Oh, yeah. -No. What game?"
3592,00:52:03.370,00:52:05.664,"Wait, what is... You mean, this is..."
3593,00:52:05.747,00:52:07.249,"No, it's just Dungeons &amp; Dragons."
